In [3]:
# SCRIPT PROCESO AM
# Genera archivos necesarios para revisión de la cartola procesada en BackOffice.
# Los archivos csv son separados en distintas carpetas para facilitar análisis contable.
import sys

import pandas as pd
import datetime
import os

username = os.getlogin()

hoy = datetime.datetime.now()
date_format = "%d.%m.%Y"
date_formatexport = "%d-%m-%Y"
date_formatcartola = "%d/%m/%Y"
fecha_actual = hoy.strftime(date_format)
fecha_cartola = hoy.strftime(date_formatexport)

df1 = pd.DataFrame()
df2 = pd.DataFrame()

folder_path = 'C:\\Users\\' + username + '\\Documents\\Py\\ARCHIVOS PROCESO AM\\'

print("Cargando archivos...")

files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
df_list = []
for file in files:
    filesh = len(files)
    df = pd.read_excel(os.path.join(folder_path, file),
                       sheet_name='Movimientos',
                       header=0,
                       names=['fecha', 'sucursal', 'ccte', 'alias', 'ncartola', 'operacion', 'descripcion',
                              'cargos',
                              'abonos', 'saldo'],
                       thousands='',
                       date_format=date_formatcartola)
    df_list.append(df)
df1 = pd.concat(df_list)

files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
df_list = []
for file in files:
    filesl = len(files)
    df = pd.read_csv(os.path.join(folder_path, file),
                     sep=';',
                     header=None,
                     thousands='.',
                     names=['fecha', 'sucursal', 'operacion', 'descripcion', 'cargos', 'abonos', 'saldo'],
                     date_format=date_formatexport)
    df_list.append(df)
df2 = pd.concat(df_list)

if filesh != 1:
    sys.exit()
if filesl != 1:
    sys.exit()

# Preparacion de datos
df1['operacion'] = df1['operacion'].astype(int)
df2['operacion'] = df2['operacion'].astype(int)
df1 = df1.drop(columns=['ccte', 'alias', 'ncartola', 'saldo'])
df2 = df2.drop(columns=['saldo'])

# Filtros por glosa exacta
fhist = [
    'Transferencia otro banco de rut 77143385-5 fintoc spa',
    'Deposito con documentos',
    'Pagos varios banco de credito e i rut 85325100-3 inversiones y ta',
    'Pagos varios banco de chile rut 76516950-k mercadopago s.a.',
    'PAGOS VARIOS TRANSBANK RUT 96689310-9 TRANSBANK S.A.',
    'RECHAZO CONVENIO PAGO PROVEEDORES',
    'Provision en linea convenio proveedores numero 0010205838 nomina 00001667094',
    'Comercio exterior',
    'Cargo transferencia de fondos por mandato cliente'
]
flinea = [
    'OTROS ABONOS',
    'RECHAZO CONVENIO PAGO PROVEEDORES',
    'DEPOSITO CON DOCUMENTOS',
    'CORRECCION DE CHEQUE MAL DIGITADO',
    'RECHAZO CONVENIO PAGO PROVEEDORES',
    'RECHAZO PAGO PROVEEDORES CONVENIO 0010205838',
    'TRANSFERENCIA MANDATO CLIENTE'
]
fb2b = [
    'Deposito con documentos',
    'Pagos varios banco de credito e i rut 85325100-3 inversiones y ta',
    'Pagos varios banco de chile rut 76516950-k mercadopago s.a.',
    'Pagos varios desde bco chile',
    'Pagos varios desde bco bci'
]
ffintoc = ['Transferencia otro banco de rut 77143385-5 fintoc spa']

# Generar dataframes abonos filtrados
cargos = df1[df1['abonos'] == 0]
b2b = df1[df1['descripcion'].isin(fb2b)]
fintoc = df1[df1['descripcion'].isin(ffintoc)]
rechazo = df1[df1['descripcion'].str.startswith('Rechazo')]

# Generar dataframe abonos tesorería
tes = cargos
tes = pd.concat([tes, rechazo])

# Limpiar cartola histórica
df1 = df1[~df1['descripcion'].isin(fhist)]
df1 = df1[~df1['descripcion'].str.startswith('Rechazo')]
df1 = df1[~df1['descripcion'].str.startswith('Provision en linea')]
df1 = df1[df1['abonos'] != 0]

# Limpiar cartola en línea
df2 = df2[~df2['abonos'].isin(fintoc['abonos'])]
df2 = df2[~df2['descripcion'].isin(flinea)]
df2 = df2[~df2['descripcion'].str.startswith('Rechazo')]
df2 = df2[df2['abonos'] != 0]

# Crear merge dataframe
df1['ind'] = 'H'
df2['ind'] = 'L'
dfc = pd.merge(df2, df1, on=['abonos', 'cargos', 'ind'], how='outer')
dfc.sort_values(by=['abonos','ind'], ascending=[True,False], inplace=True)

dfcnar = dfc.copy()
dfcna = dfc.copy()

for value in dfcna['abonos']:
    # Conteo abonos del mismo monto por glosas en cada cartola
    counth = dfcna[dfcna['ind'].str.contains("H") & (dfcna['abonos'] == value)].shape[0]
    countl = dfcna[dfcna['ind'].str.contains("L") & (dfcna['abonos'] == value)].shape[0]
    if countl > counth:
        dfcna.drop(dfcna[dfcna['abonos'] == value].index[:counth + countl], inplace=True)
    else:
        dfcna.drop(dfcna[dfcna['abonos'] == value].index[:countl * 2], inplace=True)
        dfcnar.drop(dfcnar[dfcnar['abonos'] == value].index[:counth + countl], inplace=True)

# Formato DFCNA
dfcna['sucursal_x'] = dfcna['sucursal_y']
dfcna['operacion_x'] = dfcna['operacion_y']
dfcna['descripcion_x'] = dfcna['descripcion_y']
dfcna['fecha_x'] = dfcna['fecha_y']
dfcna.rename(columns={'sucursal_x': 'sucursal'}, inplace=True)
dfcna.rename(columns={'operacion_x': 'operacion'}, inplace=True)
dfcna.rename(columns={'descripcion_x': 'descripcion'}, inplace=True)
dfcna.rename(columns={'fecha_x': 'fecha'}, inplace=True)
dfcna = dfcna.drop(columns=['sucursal_y', 'operacion_y', 'descripcion_y','fecha_y'])

# Limpiar DFCNA
dfcna = dfcna[~dfcna['abonos'].isin(b2b['abonos'])]
dfcna = dfcna[~dfcna['abonos'].isin(fintoc['abonos'])]
dfcna = dfcna[~dfcna['abonos'].isin(cargos['cargos'])]
dfcna = dfcna[~dfcna['abonos'].isin(rechazo['abonos'])]
# dfcna = dfcna[dfcna['abonos'] != 0]


Cargando archivos...


# Exportar cartolas procesadas

In [2]:

dfc.to_csv('C:\\Users\\' + username + '\\Documents\\Py\\ARCHIVOS PROCESO AM\\GENERADOS\\MERGE\\MERGE ' +
           fecha_actual + '.csv',
           sep=';',
           index=False,
           header=False,
           float_format='%.0f',
           date_format=date_formatexport)

b2b.to_csv('C:\\Users\\' + username + '\\Documents\\Py\\ARCHIVOS PROCESO AM\\GENERADOS\\B2B\\B2B ' + fecha_actual +
           '.csv',
           sep=';',
           index=False,
           header=False,
           float_format='%.0f')

fintoc.to_csv('C:\\Users\\' + username + '\\Documents\\Py\\ARCHIVOS PROCESO AM\\GENERADOS\\FINTOC\\FINTOC ' +
              fecha_actual +
              '.csv',
              sep=';',
              index=False, header=False,
              float_format='%.0f')

tes.to_csv('C:\\Users\\' + username + '\\Documents\\Py\\ARCHIVOS PROCESO AM\\GENERADOS\\TES\\TES ' + fecha_actual +
           '.csv',
           sep=';',
           index=False,
           header=False,
           float_format='%.0f')

dfcna.to_csv('C:\\Users\\' + username + '\\Documents\\Py\\ARCHIVOS PROCESO AM\\GENERADOS\\NA\\NA ' + fecha_actual +
             '.csv',
             sep=';',
             index=False,
             header=False,
             float_format='%.0f')
if dfcnar.empty:
    print("No hay registros a reversar")
else:
    dfcnar.to_csv('C:\\Users\\' + username + '\\Documents\\Py\\ARCHIVOS PROCESO AM\\GENERADOS\\REVERSAR\\REVERSA ' +
                  fecha_actual + '.csv',
                  sep=';',
                  index=False,
                  header=False,
                  float_format='%.0f')
    print("\033[1;31mRegistros a reversar. Revisar archivo generado:\033[0m")
    print("'C:\\Users\\dxs81864\\Documents\\Py\\ARCHIVOS PROCESO AM\\GENERADOS\\REVERSAR\\\033[1;31mREVERSA  " +
          fecha_actual + ".csv\033[0m'")


cartola.to_csv('C:\\Users\\' + username + '\\Documents\\Py\\ARCHIVOS PROCESO AM\\GENERADOS\\RENOMBRAR.csv',
               sep=';',
               index=False,
               header=False,
               float_format='%.0f',
               date_format=date_formatexport)

df1.to_csv('C:\\Users\\' + username + '\\Documents\\Py\\ARCHIVOS PROCESO AM\\CARTOLAS HISTORICAS\\HISTORICA ' +
           fecha_actual + '.csv',
           sep=';',
           index=False,
           header=False)

print("Archivos generados")
print("\033[1;32mRecuerde renombrar el archivo generado según la cartola en línea del día anterior: \033[0m")
print("'C:\\Users\\'" + username + "'\\Documents\\Py\\ARCHIVOS PROCESO AM\\GENERADOS\\\033[1;31mRENOMBRAR.csv\033[0m'")

No hay registros a reversar


NameError: name 'cartola' is not defined